In [ ]:
# IMPORT REQUIRED MODULES 
import os
import csv
from osgeo import gdal
import numpy as np
import datetime

In [ ]:
# Define the filepath to the ITS LIVE 6 and 12 day GeoTiffs
folder_path = 'R:/KOGE_BUGT/ITS_LIVE_6_DAY/ITS_LIVE_vel_EPSG3413_G0120_X150000_Y-2750000'

# Define the filepath for the sampled velocity CSV output file
csv_file = 'R:/KOGE_BUGT/github/data/its_live_sampled_6_12_day.csv'

# Define the X, Y coordinates of the sampling locations, extracted from QGIS (T, T5, T10, T15)
coordinates = [(183178, -2724686), (179591, -2721210), (175974, -2717759), (172309, -2714358)]

# Sample ice velocity data from the GeoTiffs at the specified coordinates and save to a CSV file
csv_headers = ['Filename', 'start_date', 'end_date', 'mid_date'] + [f'POINT_{i+1}_VELOCITY' for i in range(len(coordinates))]
try:
    with open(csv_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(csv_headers)
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.tif', '.tiff')):
                file_path = os.path.join(folder_path, filename)
                date_parts = filename.split('_')
                if len(date_parts) < 2:
                    print(f"Filename '{filename}' does not have the expected format.")
                    continue
                try:
                    start_date, end_date = date_parts[:2]
                    start_date_obj = datetime.datetime.strptime(start_date, "%Y-%m-%d")
                    end_date_obj = datetime.datetime.strptime(end_date, "%Y-%m-%d")
                    mid_date = (start_date_obj + (end_date_obj - start_date_obj) / 2).strftime("%Y/%m/%d")
                except ValueError as e:
                    print(f"Date parsing error for file '{filename}': {e}")
                    continue
                try:
                    dataset = gdal.Open(file_path, gdal.GA_ReadOnly)
                    if dataset is None:
                        raise FileNotFoundError(f"Failed to open raster file: {file_path}")
                    values = [filename, start_date, end_date, mid_date]
                    for (x, y) in coordinates:
                        try:
                            geotransform = dataset.GetGeoTransform()
                            if not geotransform:
                                raise RuntimeError("Unable to retrieve geotransform from dataset.")
                            x_pixel = int((x - geotransform[0]) / geotransform[1])
                            y_pixel = int((y - geotransform[3]) / geotransform[5])
                            if 0 <= x_pixel < dataset.RasterXSize and 0 <= y_pixel < dataset.RasterYSize:
                                band = dataset.GetRasterBand(1)
                                value = band.ReadAsArray(x_pixel, y_pixel, 1, 1)[0, 0]
                                values.append('' if np.isnan(value) else value)
                            else:
                                values.append('')
                        except Exception as e:
                            print(f"Error processing coordinate ({x}, {y}) for file '{filename}': {e}")
                            values.append('')
                    csv_writer.writerow(values)
                except Exception as e:
                    print(f"Error processing file '{filename}': {e}")

except OSError as e:
    print(f"File handling error: {e}")

print(f"Data extracted and saved to '{csv_file}'.")